### Pyspark Tutorials Mlib

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark6").getOrCreate()
spark

In [2]:
training = spark.read.csv('test4.csv',header=True,inferSchema=True)
training.show()

+-------+---+-----------+------+
|   Name|Age|Experiances|Salary|
+-------+---+-----------+------+
|Vaibhav| 31|          4| 30000|
|  Lrish| 25|          6| 15000|
|  Vikas| 29|         12| 20000|
| Gaurav| 39|          9| 60000|
| Kailas| 34|         15|  3000|
|Bhavesh| 30|          3|  1500|
|  Vidhi| 21|          6|  4000|
|  Sunny| 29|          9|  9000|
|   Paul| 24|          4|  7000|
|  Harsa| 29|          5|  6000|
|Shubham| 41|          2|  1000|
+-------+---+-----------+------+



In [3]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experiances: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
training.columns

['Name', 'Age', 'Experiances', 'Salary']

[Age,Experiances] ---> new feature ---> independ fetaure

In [5]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["Age","Experiances"],outputCol="Independent Features")

In [6]:
featureassembler

VectorAssembler_f724cd49ffc7

In [7]:
output = featureassembler.transform(training)

In [8]:
output.show()

+-------+---+-----------+------+--------------------+
|   Name|Age|Experiances|Salary|Independent Features|
+-------+---+-----------+------+--------------------+
|Vaibhav| 31|          4| 30000|          [31.0,4.0]|
|  Lrish| 25|          6| 15000|          [25.0,6.0]|
|  Vikas| 29|         12| 20000|         [29.0,12.0]|
| Gaurav| 39|          9| 60000|          [39.0,9.0]|
| Kailas| 34|         15|  3000|         [34.0,15.0]|
|Bhavesh| 30|          3|  1500|          [30.0,3.0]|
|  Vidhi| 21|          6|  4000|          [21.0,6.0]|
|  Sunny| 29|          9|  9000|          [29.0,9.0]|
|   Paul| 24|          4|  7000|          [24.0,4.0]|
|  Harsa| 29|          5|  6000|          [29.0,5.0]|
|Shubham| 41|          2|  1000|          [41.0,2.0]|
+-------+---+-----------+------+--------------------+



In [9]:
output.columns

['Name', 'Age', 'Experiances', 'Salary', 'Independent Features']

In [10]:
finalized_data = output.select("Independent Features","Salary")

In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [31.0,4.0]| 30000|
|          [25.0,6.0]| 15000|
|         [29.0,12.0]| 20000|
|          [39.0,9.0]| 60000|
|         [34.0,15.0]|  3000|
|          [30.0,3.0]|  1500|
|          [21.0,6.0]|  4000|
|          [29.0,9.0]|  9000|
|          [24.0,4.0]|  7000|
|          [29.0,5.0]|  6000|
|          [41.0,2.0]|  1000|
+--------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression

# train test split

train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor = regressor.fit(train_data)

In [13]:
### Coefficients

regressor.coefficients

DenseVector([-230.2158, -278.2258])

In [14]:
### Intercepts

regressor.intercept

16922.352440628223

In [15]:
#### Prediction

pred_results = regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|         [29.0,12.0]| 20000|6907.383770403034|
|          [39.0,9.0]| 60000|5439.902916376552|
+--------------------+------+-----------------+



In [17]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(33826.35665661021, 1574110396.754961)